In [2]:
import os
from pathlib import Path
import shutil

import arcpy
from gtfs_tools.gtfs import GtfsDataset

In [3]:
dir_prj = Path.cwd().parent
dir_data = dir_prj / 'data'

dir_raw = dir_data / 'raw'
dir_int = dir_data / 'interim'

gdb_int = dir_int / 'interim.gdb'

dir_gtfs_parent = dir_raw / 'gtfs_olympia'

# ensure can overwrite output for each run
arcpy.env.overwriteOutput = True

# where to find the network dataset
network_dataset = Path(r"D:\data\ba_data\usa_2024\Data\StreetMap Premium Data\northamerica.geodatabase\main.Routing\main.Routing_ND")

In [4]:
# create a location to stage data to
tmp_gtfs = dir_int / dir_gtfs_parent.name

if not tmp_gtfs.exists():
    tmp_gtfs.mkdir(parents=True)

In [5]:
# create a list of tuples, the source and temporary location for cleaned up data
source_lst = [(pth.parent, tmp_gtfs / pth.parent.name) for pth in dir_gtfs_parent.glob('**/shapes.txt')]

source_lst

[(WindowsPath('D:/projects/gtfs-tools/data/raw/gtfs_olympia/mdb_source_id=11'),
  WindowsPath('D:/projects/gtfs-tools/data/interim/gtfs_olympia/mdb_source_id=11')),
 (WindowsPath('D:/projects/gtfs-tools/data/raw/gtfs_olympia/mdb_source_id=1330'),
  WindowsPath('D:/projects/gtfs-tools/data/interim/gtfs_olympia/mdb_source_id=1330')),
 (WindowsPath('D:/projects/gtfs-tools/data/raw/gtfs_olympia/mdb_source_id=1331'),
  WindowsPath('D:/projects/gtfs-tools/data/interim/gtfs_olympia/mdb_source_id=1331')),
 (WindowsPath('D:/projects/gtfs-tools/data/raw/gtfs_olympia/mdb_source_id=263'),
  WindowsPath('D:/projects/gtfs-tools/data/interim/gtfs_olympia/mdb_source_id=263')),
 (WindowsPath('D:/projects/gtfs-tools/data/raw/gtfs_olympia/mdb_source_id=264'),
  WindowsPath('D:/projects/gtfs-tools/data/interim/gtfs_olympia/mdb_source_id=264')),
 (WindowsPath('D:/projects/gtfs-tools/data/raw/gtfs_olympia/mdb_source_id=265'),
  WindowsPath('D:/projects/gtfs-tools/data/interim/gtfs_olympia/mdb_source_id=265'

In [6]:
# list to populate
fc_lst = []

In [7]:
for source in source_lst:
    
    # temporary locations for intermediate data
    lines_fc = gdb_int / "lines_tmp"
    stops_fc = gdb_int / "stops_tmp"

    # start by copying the orignal GTFS dataset to a temporary location
    shutil.copytree(source[0], source[1], dirs_exist_ok=True)

    # path for new trips, remove since will be replaced
    trips_pth = source[1] / "trips.txt"
    trips_pth.unlink(missing_ok=True)

    # match to geometries
    arcpy.transit.GenerateShapesFeaturesFromGTFS(
        in_gtfs_folder=str(source[0]),
        out_shape_lines=str(lines_fc),
        out_shape_stops=str(stops_fc),
        out_gtfs_trips=str(trips_pth),
        network_modes="0;1;2;3;4;5;6;7;11;12;OTHER",
        network_data_source=str(network_dataset),
        travel_mode="Rural Driving Time",
        drive_side="RIGHT",
        bearing_tolerance=30,
        max_bearing_angle=65
    )

    # paths for shapes and stop times
    shapes_pth = source[1] / 'shapes.txt'
    stop_times_pth = source[1] / 'stop_times.txt'

    # remove targets to be saved
    shapes_pth.unlink(missing_ok=True)
    stop_times_pth.unlink(missing_ok=True)

    # save updated versions
    arcpy.transit.FeaturesToGTFSShapes(
        in_shape_lines=str(lines_fc),
        in_shape_stops=str(stops_fc),
        in_gtfs_trips=str(trips_pth),
        in_gtfs_stop_times=str(source[0] / 'stop_times.txt'),
        out_gtfs_shapes=str(shapes_pth),
        out_gtfs_stop_times=str(stop_times_pth)
    )
    
    # create the gtfs object insance
    gtfs = GtfsDataset(source[1])
    
    # get the trips from the GTFS dataset
    trips = gtfs.trips

    # create a path to save the data
    fc_name = f'mdb_source_{gtfs.gtfs_folder.stem.split("=")[1]}_trips'
    fc_pth = gdb_int / fc_name
    
    # get a trips data frame with reduced columns but including agency information
    trips_sedf = (gtfs.trips.sedf.loc[trips.sedf['SHAPE'].notnull(), ['route_id', 'trip_id', 'trip_headsign', 'SHAPE']]
                  .merge(gtfs._crosstab_route_agency, on='route_id')
                  .merge(gtfs.agency.data.loc[:,['agency_id', 'agency_name']], on='agency_id')
                 )

    # save the features
    trips_sedf.spatial.to_featureclass(fc_pth)
    
    # add the path to the list
    fc_lst.append(fc_pth)
    
    print(fc_pth)

D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_11_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_1330_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_1331_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_263_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_264_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_265_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_266_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_267_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_268_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_283_trips
D:\projects\gtfs-tools\data\interim\interim.gdb\mdb_source_735_trips


ExecuteError: Traceback (most recent call last):
  File "c:\program files\arcgis\pro\Resources\ArcToolbox\scripts\FeaturesToGTFSShapes.py", line 27, in FeaturesToGTFSShapes
    tool.execute()
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcToolbox\Scripts\features_to_gtfs_shapes.py", line 101, in execute
    self._write_stop_times_with_shape_dist_traveled()
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcToolbox\Scripts\features_to_gtfs_shapes.py", line 436, in _write_stop_times_with_shape_dist_traveled
    self._calculate_stops_shape_dist_traveled()
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcToolbox\Scripts\features_to_gtfs_shapes.py", line 276, in _calculate_stops_shape_dist_traveled
    sequence_stopid_dict, sequence_geom_dict = self._get_stop_info_for_shape(shape_id)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcToolbox\Scripts\features_to_gtfs_shapes.py", line 232, in _get_stop_info_f

In [95]:
trips_fc = str(gdb_int / 'trips')

if arcpy.Exists(trips_fc):
    arcpy.management.Delete(trips_fc)

In [96]:
arcpy.management.Merge(
    inputs=';'.join(map(str, fc_lst)), 
    output=str(gdb_int / 'trips')
)

In [ ]:
arcpy.edit.Snap(
    in_features=trips_fc,
    snap_environment=[[trips_fc, "EDGE", "30 Feet"]]
)

In [ ]:
arcpy.transit.GenerateShapesFeaturesFromGTFS(
    in_gtfs_folder=r"D:\projects\gtfs-tools\data\raw\gtfs_olympia\mdb_source_id=268",
    out_shape_lines=r"memory\mdb_268_lines",
    out_shape_stops=r"memory\mdb_268_stops",
    out_gtfs_trips=r"D:\projects\gtfs-tools\data\interim\mdb_268_trips.txt",
    network_modes="0;1;2;3;4;5;6;7;11;12;OTHER",
    network_data_source=r"D:\data\ba_data\usa_2024\Data\StreetMap Premium Data\northamerica.geodatabase\main.Routing\main.Routing_ND",
    travel_mode="Rural Driving Time",
    drive_side="RIGHT",
    bearing_tolerance=30,
    max_bearing_angle=65
)